## 1. Dependencies, Imports & Setup

In [ ]:
!apt-get -qq install poppler-utils tesseract-ocr
%pip install -q --user --upgrade pillow
%pip install -q --upgrade unstructured["all-docs"]
%pip install pymupdf
%pip install kdbai_client
%pip install langchain-openai
%pip install langchain
%pip install langchain-community
%pip install --upgrade nltk

In [2]:
from unstructured.partition.pdf import partition_pdf
from unstructured.partition.auto import partition
from unstructured.embed.openai import OpenAIEmbeddingConfig, OpenAIEmbeddingEncoder
import fitz
from langchain_openai import OpenAIEmbeddings
# import kdbai_client as kdbai
from langchain_community.vectorstores import KDBAI
from langchain.chains import RetrievalQA
from langchain_openai import ChatOpenAI
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     /home/alexandre_pantalena/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

## 2. Use Unstructured to Process Complex PDF Documentation

1. Read in data
2. Partition using the 'hi_res' strategy
3. Chunk

In [3]:
!mkdir './tables'

In [4]:
file = '42888360000111-REL24092024V01-000743017.pdf'

elements = partition_pdf(file,
                              strategy="hi_res",
                              chunking_strategy="by_title",
                              max_characters=2500,
                              new_after_n_chars=2300,
                              )

### Explore the extracted elements

In [5]:
from collections import Counter
display(Counter(type(element) for element in elements))

Counter({unstructured.documents.elements.CompositeElement: 18,
         unstructured.documents.elements.Table: 5})

In [6]:
for element in elements:
  print(type(element))

<class 'unstructured.documents.elements.CompositeElement'>
<class 'unstructured.documents.elements.Table'>
<class 'unstructured.documents.elements.CompositeElement'>
<class 'unstructured.documents.elements.Table'>
<class 'unstructured.documents.elements.CompositeElement'>
<class 'unstructured.documents.elements.Table'>
<class 'unstructured.documents.elements.CompositeElement'>
<class 'unstructured.documents.elements.Table'>
<class 'unstructured.documents.elements.CompositeElement'>
<class 'unstructured.documents.elements.Table'>
<class 'unstructured.documents.elements.CompositeElement'>
<class 'unstructured.documents.elements.CompositeElement'>
<class 'unstructured.documents.elements.CompositeElement'>
<class 'unstructured.documents.elements.CompositeElement'>
<class 'unstructured.documents.elements.CompositeElement'>
<class 'unstructured.documents.elements.CompositeElement'>
<class 'unstructured.documents.elements.CompositeElement'>
<class 'unstructured.documents.elements.CompositeEle

In [12]:
for element in elements:
  if element.to_dict()['type'] == 'Table':
    print(element.text)

R$ 0,100 Por cota 1,18%(1) Dividend yield 136,6% Dividend Yield % CDI 160,5% % CDI Gross Up IR 15,15%(2) Dividend Yield 12M CDI + 5,1% 59,2% da carteira IPCA + 11,7% 20,6% da carteira 0% exposição direta em produtor rural (PF) 100% adimplente Carteira Total 83,9% Alocado em ativos de crédito do agronegócio 18.150 Investidores
CDI IPCA Mês 0,87% -0,02% 12 Meses 11,20% 4,24%
ago/24 jul/24 jun/24 mai/24 abr/24 mar/24 fev/24 Receitas Totais 755.320 779.536 694.144 767.304 789.198 766.329 718.070 Juros 645.471 672.997 594.972 643.609 671.017 634.427 614.527 Correção Monetária 39.423 32.546 41.541 44.197 53.130 68.001 52.169 Rendimento do Caixa 70.425 73.993 57.631 53.430 65.051 53.164 48.373 Outras Receitas - - - 26.068 - 10.739 3.002 Despesas Totais (73.517) (105.371) (69.263) (83.740) (67.087) (64.291) (67.567) Despesas Operacionais Provisões (1) (73.517) (105.371) (69.263) (83.740) (67.087) (64.291) (67.567) - - - - - - - Reserva de Lucros (2) (7.913) (274) 183.788 57.715 (48.221) (28.14

### What a table element looks like after extraction:

In [13]:
print(elements[-2])

investidor ao aplicar seus recursos.


### Embed Extracted Elements with OpenAI Embedding Model